In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.max_rows = 100

## Feature engineering

In [ ]:
# Creating the datasets
house_data_train = pd.read_csv("train.csv")
house_data_test = pd.read_csv("test.csv")
# Target prices:
y = house_data_train['SalePrice']
house_data_train.info()

In [ ]:
# Columns I will work with
columns = [
    'MSSubClass',
    'LotArea',
    'LotShape',
    'LandContour',
    'Utilities',
    'LandSlope',
    'Condition1',
    'Condition2',
    'BldgType',
    'HouseStyle',
    'OverallQual',
    'OverallCond',
    'Foundation',
    '1stFlrSF',
    '2ndFlrSF',
    'BedroomAbvGr',
    'KitchenAbvGr',
    'FullBath',
    'HalfBath',
    'GarageArea',
    'PoolArea'
]

In [ ]:
categorical_columns = list(house_data_train.dtypes[house_data_train.dtypes == np.object].index)
numerical_columns = list(house_data_train.dtypes[house_data_train.dtypes != np.object].index)

In [ ]:
# Numerical and categorical dataframes
numerical_train = house_data_train[numerical_columns]
numerical_test = house_data_test[numerical_columns]
categorical_train = house_data_train[categorical_columns]
categorical_test = house_data_test[categorical_columns]

In [ ]:
# Values and meanings from the first column, MSSubClass
MSSubClass_dict = { 
        20:	'1-STORY 1946 & NEWER ALL STYLES', 
        30:	'1-STORY 1945 & OLDER', 
        40:	'1-STORY W/FINISHED ATTIC ALL AGES',
        45:	'1-1/2 STORY - UNFINISHED ALL AGES', 
        50:	'1-1/2 STORY FINISHED ALL AGES', 
        60:	'2-STORY 1946 & NEWER', 
        70:	'2-STORY 1945 & OLDER', 
        75:	'2-1/2 STORY ALL AGES', 
        80:	'SPLIT OR MULTI-LEVEL', 
        85:	'SPLIT FOYER', 
        90:	'DUPLEX - ALL STYLES AND AGES', 
       120:	'1-STORY PUD (Planned Unit Development) - 1946 & NEWER', 
       150:	'1-1/2 STORY PUD - ALL AGES', 
       160:	'2-STORY PUD - 1946 & NEWER', 
       180:	'PUD - MULTILEVEL - INCL SPLIT LEV/FOYER', 
       190:	'2 FAMILY CONVERSION - ALL STYLES AND AGES'}

# Replacing numbers with column names
categorical_train['MSSubClass'].replace(MSSubClass_dict, inplace=True)
categorical_test['MSSubClass'].replace(MSSubClass_dict, inplace=True)

In [ ]:
# Meanings of the Condition1 and Condition2 column values
cond = {
    'Artery':	'Adjacent to arterial street',
    'Feedr':	'Adjacent to feeder street',	
    'Norm':	'Normal',	
    'RRNn':	'Within 200 of North-South Railroad',
    'RRAn':	'Adjacent to North-South Railroad',
    'PosN':	'Near positive off-site feature--park, greenbelt, etc.',
    'PosA':	'Adjacent to postive off-site feature',
    'RRNe':	'Within 200 of East-West Railroad',
    'RRAe':	'Adjacent to East-West Railroad'
}
# Getting the full name of the conditions
categorical_train['Condition1'].replace(cond, inplace=True)
categorical_train['Condition1'].replace(cond, inplace=True)
categorical_test['Condition2'].replace(cond, inplace=True)
categorical_test['Condition2'].replace(cond, inplace=True)

In [ ]:
def sum_1hots(df, column1, column2):
    '''This function is for getting only one hot encoder for repeated 
    conditions in some columns in the dataset '''
    c1 = pd.get_dummies(df[column1])
    c2 = pd.get_dummies(df[column2])
    # Sum both one hot encoders. Exploring this data, I found some 2's, that makes 
    # no sense, so, I replaced them with 1's
    c1c2 = c1.add(c2, fill_value=0).replace(2,1)
    df = df.drop(columns=[column1, column2])
    df = pd.concat([df, c1c2], axis=1)
    return df

In [ ]:
categorical_train = sum_1hots(categorical_train, 'Condition1', 'Condition2')
categorical_train = sum_1hots(categorical_train, 'Exterior1st', 'Exterior2nd')
categorical_test = sum_1hots(categorical_test, 'Condition1', 'Condition2')
categorical_test = sum_1hots(categorical_test, 'Exterior1st', 'Exterior2nd')

In [ ]:
def dummies(df, column):
    '''This function accepts a dataframe and a column name. It will get the 
    one hot encoder for categorical column, then, drop it from the original
    dataframe and cancatenate the one hot encoder'''
    df1 = pd.get_dummies(df[column])
    df = df.drop(columns=[column])
    df = pd.concat([df1, df], axis=1)
    return df

In [ ]:
# Using the custom function in the left categorical data columns.
partial_train= dummies(partial_train, 'BldgType')
partial_train= dummies(partial_train, 'HouseStyle')
partial_train= dummies(partial_train, 'Foundation')
partial_train= dummies(partial_train, 'LotShape')
partial_train= dummies(partial_train, 'LandContour')
partial_train= dummies(partial_train, 'LandSlope')
# partial_train= dummies(partial_train, 'Utilities')

partial_test= dummies(partial_test, 'BldgType')
partial_test= dummies(partial_test, 'HouseStyle')
partial_test= dummies(partial_test, 'Foundation')
partial_test= dummies(partial_test, 'LotShape')
partial_test= dummies(partial_test, 'LandContour')
partial_test= dummies(partial_test, 'LandSlope')
# partial_test= dummies(partial_test, 'Utilities')

## Normalizing the data

In [ ]:
from sklearn import preprocessing
#Train set
x_train = partial_train.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_train_scaled = min_max_scaler.fit_transform(x_train)

# Test set
x_test = partial_test.values #returns a numpy array
x_test_scaled = min_max_scaler.fit_transform(x_test)
# Filling NaN values with 0
x_test_scaled[np.isnan(x_test_scaled)] = 0

### Correlations 

In [ ]:
norm_train = pd.DataFrame(data = x_train_scaled, columns=partial_train.columns)
norm_train = pd.concat([norm_train, y], axis=1)
norm_train

In [ ]:
norm_train.max()

In [ ]:
corr_matrix = norm_train.corr()
(corr_matrix['SalePrice']
.sort_values(ascending=False)
.where(((corr_matrix['SalePrice'] < -0.05) | (corr_matrix['SalePrice'] > 0.05)), other=np.nan)
.dropna()
)

## Machine Learning Models

In [ ]:
from sklearn import svm

regr = svm.SVR()
regr.fit(x_train_scaled, y.values)
predictions_train = regr.predict(x_train_scaled)
predictions = regr.predict(x_test_scaled)
predictions

In [ ]:
np.max(predictions_train / y.values)

In [ ]:
import csv
j = 1461
with open('predictions.csv', 'w') as p:
    writer = csv.writer(p)
    writer.writerow(['Id','SalePrice'])
    for pred in predictions:
        writer.writerow([j,pred])
        j += 1



In [ ]:
from sklearn.linear_model import SGDRegressor
sgd = SGDRegressor(max_iter=5000, learning_rate='adaptive')
sgd.fit(x_train_scaled, y.values)
predictions_sgd_train = sgd.predict(x_train_scaled)
predictions_sgd = sgd.predict(x_test_scaled)
predictions_sgd

In [ ]:
predictions_sgd_train / y.values

In [ ]:
import csv
j = 1461
with open('predictions2.csv', 'w') as p:
    writer = csv.writer(p)
    writer.writerow(['Id','SalePrice'])
    for pred in predictions_sgd:
        writer.writerow([j,pred])
        j += 1

In [46]:
categorical_columns = list(house_data_train.dtypes[house_data_train.dtypes == np.object])

MSSubClass       object
MSZoning         object
Street           object
Alley            object
LotShape         object
LandContour      object
Utilities        object
LotConfig        object
LandSlope        object
Neighborhood     object
Condition1       object
Condition2       object
BldgType         object
HouseStyle       object
RoofStyle        object
RoofMatl         object
Exterior1st      object
Exterior2nd      object
MasVnrType       object
ExterQual        object
ExterCond        object
Foundation       object
BsmtQual         object
BsmtCond         object
BsmtExposure     object
BsmtFinType1     object
BsmtFinType2     object
Heating          object
HeatingQC        object
CentralAir       object
Electrical       object
KitchenQual      object
Functional       object
FireplaceQu      object
GarageType       object
GarageFinish     object
GarageQual       object
GarageCond       object
PavedDrive       object
PoolQC           object
Fence            object
MiscFeature     